In [1]:
%matplotlib inline
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import random
import xgboost as xgb

from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from xgboost import XGBRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import matthews_corrcoef, roc_auc_score
from sklearn.grid_search import RandomizedSearchCV
from catboost import CatBoostClassifier

from rgf.sklearn import RGFClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression,LinearRegression

from sklearn.ensemble import ExtraTreesClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from xgboost import XGBRegressor

C:\Users\royal\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\royal\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
%%time
os.chdir('C:\\Users\\royal\\Downloads\\Compressed\\flight_predictor_data\\oof+test')

Wall time: 0 ns


In [ ]:
os.listdir()

In [160]:
%%time
os.chdir('C:\\Users\\royal\\Downloads\\Compressed\\flight_predictor_data')
train=pd.read_csv('weather_data_train.csv')
test=pd.read_csv('weather_data_test.csv')
train_flt = pd.read_csv('flight_data_train.csv')

Wall time: 8.4 s


In [162]:
#generate the output total and labels.
train_flt['total']=0
for i in range(1,289):
    train_flt['total']+=train_flt['Spot'+str(i)+' totalFlights']

train_flt['label']=0
for i,j in enumerate(train_flt['total']):
    if j>=15:
        train_flt.loc[i, 'label']=1

In [154]:
Y=train_flt['label'].values

In [156]:
train11 = pd.DataFrame()
test11 = pd.DataFrame()

In [157]:
##get the difference between top and lowerheight parameters.
aa=['Dew Point','Pressure','Temperature','Wind Direction','Wind Speed']
hh=['Station1','Station2','Station3','Station4','Station5']
for i in aa:
    for j in hh:
        train11[j+' '+i+' Height Diff']=train[j+' '+i+' Height45']-train[j+' '+i+' Height1']
        test11[j+' '+i+' Height Diff']=test[j+' '+i+' Height45']-test[j+' '+i+' Height1']

##get the max,min,standard deviation,variance,mean and diff for all the conditions
aa=['Dew Point','Pressure','Temperature','Wind Direction','Wind Speed']
hh=['Station1','Station2','Station3','Station4','Station5']
for i in aa:
    for j in hh:
        ls=[]
        for k in range(1,46):
            ls.append(j+' '+i+' Height'+str(k))
        train11[j+' '+i+' Height max']=train[ls].max(axis=1)
        train11[j+' '+i+' Height min']=train[ls].min(axis=1)
        train11[j+' '+i+' Height std']=train[ls].std(axis=1)
        train11[j+' '+i+' Height var']=train[ls].var(axis=1)
        train11[j+' '+i+' Height var']=train[ls].mean(axis=1)
        
        test11[j+' '+i+' Height var']=test[ls].mean(axis=1)
        test11[j+' '+i+' Height var']=test[ls].var(axis=1)
        test11[j+' '+i+' Height std']=test[ls].std(axis=1)
        test11[j+' '+i+' Height max']=test[ls].max(axis=1)
        test11[j+' '+i+' Height min']=test[ls].min(axis=1)
        
        train11[j+' '+i+' Height max-min']=train11[j+' '+i+' Height max']-train11[j+' '+i+' Height min']
        test11[j+' '+i+' Height max-min']=test11[j+' '+i+' Height max']-test11[j+' '+i+' Height min']

In [54]:
##test and train binary predictions
##test
os.chdir('C:\\Users\\royal\\Downloads\\Compressed\\flight_predictor_data\\oof+test')
cat_test = pd.read_csv('bin_cat_1150d_0.76519_test.csv')
ext_test = pd.read_csv('bin_extra_0.77593_test.csv')
ext1455_test = pd.read_csv('bin_extra_1455d_0.77927_test.csv')
ext251_test = pd.read_csv('bin_extra_251d_0.77202_test.csv')
lgb1151_test = pd.read_csv('bin_lgb_1151d_0.79085_test.csv')
lgb1201_test = pd.read_csv('bin_lgb_1201d_0.79777_test.csv')
lgb225_test = pd.read_csv('bin_lgb_225d_0.78732_test.csv')
lgbaae_test = pd.read_csv('bin_lgb_aae_0.78894_test.csv')
lgbpca_test = pd.read_csv('bin_lgb_pca_0.77974_test.csv')
lr_test = pd.read_csv('bin_lr_225d_0.77011_test.csv')
nnaae_test = pd.read_csv('bin_nn_aae_0.75009_test.csv')
rgf225_test = pd.read_csv('bin_rgf_225d_0.74622_test.csv')

xgb1450_test = pd.read_csv('bin_xgb_1450d_0.79268_test.csv')
xgbaae_test = pd.read_csv('bin_xgb_aae_0.79059_test.csv')
xgbtune_test = pd.read_csv('bin_xgb_tune_0.79425_test.csv')
stac_test = pd.read_csv('submission_stack_152.csv')

##train 
cat_train = pd.read_csv('bin_cat_1150d_0.76519_train.csv')
ext_train = pd.read_csv('bin_extra_0.77593_train.csv')
ext1455_train = pd.read_csv('bin_extra_1455d_0.77927_train.csv')
ext251_train = pd.read_csv('bin_extra_251d_0.77202_train.csv')
lgb1151_train = pd.read_csv('bin_lgb_1151d_0.79085_train.csv')
lgb1201_train = pd.read_csv('bin_lgb_1201d_0.79777_train.csv')
lgb225_train = pd.read_csv('bin_lgb_225d_0.78732_train.csv')
lgbaae_train = pd.read_csv('bin_lgb_aae_0.78894_train.csv')
lgbpca_train = pd.read_csv('bin_lgb_pca_0.77974_train.csv')
lr_train = pd.read_csv('bin_lr_225d_0.77011_train.csv')
nnaae_train = pd.read_csv('bin_nn_aae_0.75009_train.csv')
rgf225_train = pd.read_csv('bin_rgf_225d_0.74622_train.csv')
xgb1450_train = pd.read_csv('bin_xgb_1450d_0.79268_train.csv')
xgbaae_train = pd.read_csv('bin_xgb_aae_0.79059_train.csv')
xgbtune_train = pd.read_csv('bin_xgb_tune_0.79425_train.csv')
sub_146 = pd.read_csv('submission_146.csv')

ts_PERID = cat_test['Day_Id']

In [158]:
##test and train probability predictions
##test
os.chdir('C:\\Users\\royal\\Downloads\\Compressed\\flight_predictor_data\\oof+test')
cat_test = pd.read_csv('prob_cat_1150d_0.76519_test.csv')
ext_test = pd.read_csv('prob_extra_0.77593_test.csv')
ext1455_test = pd.read_csv('prob_extra_1455d_0.77927_test.csv')
ext251_test = pd.read_csv('prob_extra_251d_0.77202_test.csv')
lgb1151_test = pd.read_csv('prob_lgb_1151d_0.79085_test.csv')
lgb225_test = pd.read_csv('prob_lgb_225d_0.78732_test.csv')
lgbaae_test = pd.read_csv('prob_lgb_aae_0.78894_test.csv')
lr_test = pd.read_csv('prob_lr_225d_0.77011_test.csv')
nnaae_test = pd.read_csv('prob_nn_aae_0.75009_test.csv')
rgf225_test = pd.read_csv('prob_rgf_225d_0.74622_test.csv')
xgb1450_test = pd.read_csv('prob_xgb_1450d_0.79268_test.csv')
xgbaae_test = pd.read_csv('prob_xgb_aae_0.79059_test.csv')
xgbtune_test = pd.read_csv('prob_xgb_tune_0.79425_test.csv')

##train 
cat_train = pd.read_csv('prob_cat_1150d_0.76519_train.csv')
ext_train = pd.read_csv('prob_extra_0.77593_train.csv')
ext1455_train = pd.read_csv('prob_extra_1455d_0.77927_train.csv')
ext251_train = pd.read_csv('prob_extra_251d_0.77202_train.csv')
lgb1151_train = pd.read_csv('prob_lgb_1151d_0.79085_train.csv')
lgb225_train = pd.read_csv('prob_lgb_225d_0.78732_train.csv')
lgbaae_train = pd.read_csv('prob_lgb_aae_0.78894_train.csv')
lr_train = pd.read_csv('prob_lr_225d_0.77011_train.csv')
nnaae_train = pd.read_csv('prob_nn_aae_0.75009_train.csv')
rgf225_train = pd.read_csv('prob_rgf_225d_0.74622_train.csv')
xgb1450_train = pd.read_csv('prob_xgb_1450d_0.79268_train.csv')
xgbaae_train = pd.read_csv('prob_xgb_aae_0.79059_train.csv')
xgbtune_train = pd.read_csv('prob_xgb_tune_0.79425_train.csv')

ts_PERID = cat_test['Day_Id']

In [29]:
##average blend. 
preds1=(ext_test['Good_Bad']+lgbpca_test['Good_Bad']+xgb1450_test['Good_Bad']+lgb1151_test['Good_Bad'])/4

prediction_rfc=list(range(len(preds1)))
for i in range(len(preds1)):
    prediction_rfc[i]=1 if preds1[i]>=0.25 else 0

sub = pd.DataFrame({'Day_Id': ts_PERID, 'Good_Bad': prediction_rfc})
sub=sub.reindex(columns=["Day_Id","Good_Bad"])
filename = 'submission.csv'
sub.to_csv(filename, index=False)

In [56]:
for k,i,j in zip(cat_test['Day_Id'],lgbpca_test['Good_Bad'],lgb1151_test['Good_Bad']):
    if i!=j:
        print(k,i,j)
#         ext_test.loc[ext_test['Day_Id'] == k, 'Good_Bad'] = 1

216 0 1
380 1 0
908 0 1
1335 1 0
1586 1 0
1711 1 0
1940 1 0
2300 1 0


In [43]:
#number of 0 and 1 prediction in the binary file.
su = ext1455_test
print(set(su['Good_Bad']))
prediction_rfc=list(range(len(su['Good_Bad'])))
i0=0
i1=0
for j,i in enumerate(su['Good_Bad']):
    if i==0:
        prediction_rfc[j]=0
        i0=i0+1
    else:
        i1=i1+1
        prediction_rfc[j]=1
print(i0,i1)

{0, 1}
49 51


In [1]:
# aa=[369,379,1335,1940,1960] #0
# bb=[908,1781] #1
# aa=[1940,1960,908,1781,216,1931]
# bb=[1335,379]

In [58]:
su = lgbpca_test
for k,i in zip(cat_test['Day_Id'],su['Good_Bad']):
    if k in aa:
        su.loc[su['Day_Id'] == k, 'Good_Bad'] = 0
    if k in bb:
        su.loc[su['Day_Id'] == k, 'Good_Bad'] = 1

In [59]:
filename = 'subm.csv'
su.to_csv(filename, index=False)

In [106]:
##correlation between 2 files.
corr1[1].corr(corr1[8])

0.8070413941267186

In [163]:
train_day=train['Day_Id']
train1=train.drop(['Day_Id'],axis=1)
train1=train1.values
Y=train_flt['label'].values

test_day=test['Day_Id']
test1=test.drop(['Day_Id'],axis=1)
test1=test1.values

In [159]:
#stack level1 predictions
X_ts=np.column_stack((cat_test['Good_Bad'],ext_test['Good_Bad'],ext1455_test['Good_Bad'],lgb1151_test['Good_Bad'],lgb225_test['Good_Bad'],lr_test['Good_Bad'],nnaae_test['Good_Bad'],rgf225_test['Good_Bad'],xgb1450_test['Good_Bad'],xgbaae_test['Good_Bad'],xgbtune_test['Good_Bad']))
X_tr=np.column_stack((cat_train['Good_Bad'],ext_train['Good_Bad'],ext1455_train['Good_Bad'],lgb1151_train['Good_Bad'],lgb225_train['Good_Bad'],lr_train['Good_Bad'],nnaae_train['Good_Bad'],rgf225_train['Good_Bad'],xgb1450_train['Good_Bad'],xgbaae_train['Good_Bad'],xgbtune_train['Good_Bad']))

In [164]:
X_tr1=np.column_stack((X_tr,train11.values,train1))
X_ts1=np.column_stack((X_ts,test11.values,test1))

In [165]:
print(X_ts.shape)
print(X_tr1.shape)
print(train11.shape)
print(test11.shape)

(100, 11)
(2183, 1286)
(2183, 150)
(100, 150)


In [166]:
from sklearn.cross_validation import StratifiedKFold
SEED=42
clf = lgb.LGBMClassifier()
st_train = X_tr1
st_test = X_ts1
# clf = lgb.LGBMClassifier(max_depth= 10, learning_rate=0.044, n_estimators=255, num_leaves= 17, reg_alpha=1.0824, reg_lambda= 0.0386)
# clf=CatBoostClassifier(iterations=50)
# clf = XGBClassifier()
# clf=ExtraTreesClassifier(n_estimators=10000, criterion='entropy', max_depth=9,  min_samples_leaf=1,  n_jobs=30, random_state=1)
# clf = XGBClassifier(gamma = 1.0,learning_rate = 0.010,max_depth = 5,min_child_weight = 10,n_estimators = 338,subsample = 0.800,colsample_bytree = 0.50)
# clf = RGFClassifier(max_leaf=500,algorithm="RGF",test_interval=100, loss="LS")
# clf = LogisticRegression()

fold = 5
cv = StratifiedKFold(Y, n_folds=fold,shuffle=True, random_state=42)
X_preds = np.zeros(st_train.shape[0])
preds = np.zeros(st_test.shape[0])
for i, (tr, ts) in enumerate(cv):
    print(ts.shape)
    mod = clf.fit(st_train[tr], Y[tr])
    X_preds[ts] = mod.predict_proba(st_train[ts])[:,1]
    preds += mod.predict_proba(st_test)[:,1]
    print("fold {}, ROC AUC: {:.3f}".format(i, roc_auc_score(Y[ts], X_preds[ts])))
score = roc_auc_score(Y, X_preds)
print(score)
preds1 = preds/fold

(437,)
fold 0, ROC AUC: 0.807
(437,)
fold 1, ROC AUC: 0.785
(437,)
fold 2, ROC AUC: 0.808
(437,)
fold 3, ROC AUC: 0.779
(435,)
fold 4, ROC AUC: 0.781
0.7914202003610796


In [167]:
# pick the best threshold out-of-fold
thresholds = np.linspace(0.01, 0.99, 50)
mcc = np.array([roc_auc_score(Y, X_preds>thr) for thr in thresholds])
best_threshold = thresholds[mcc.argmax()]
print(mcc.max())
print(best_threshold)

0.7301163632835377
0.35000000000000003


In [168]:
#prediction file generation
prediction_rfc=list(range(len(preds1)))
for i in range(len(preds1)):
    prediction_rfc[i]=1 if preds1[i]>best_threshold else 0

sub = pd.DataFrame({'Day_Id': ts_PERID, 'Good_Bad': prediction_rfc})
sub=sub.reindex(columns=["Day_Id","Good_Bad"])
filename = 'submission_stack.csv'
sub.to_csv(filename, index=False)